### Creating wordpress user and wordpress database

In [ ]:
# cat /home/ec2-user/mariadb-wordpress.vars
---
mysql_root_pwd: "mysqlroot@123*"
mysql_extra_user: "wpuser"
mysql_extra_pwd: "wppwd@123*"
mysql_extra_db: "wordpressdb"

In [ ]:
# cat wordpress.yml 
---
- name: "Installing and configuring mariadb server for wordpress"
  hosts: amazon
  become: yes
  vars_files:
    - /home/ec2-user/mariadb-wordpress.vars
  tasks:
    - name: "installing mariadb and pip packages"
      yum: 
        name: 
          - mariadb-server
          - pip 
        state: present
    - name:
      pip:
        name: PyMySQL
        state: present

    - name: "Restarting/enabling mariadb server"
      service:
        name: mariadb
        state: restarted
        enabled: yes    

    - name: "Resetting root password"
      ignore_errors: yes
      mysql_user:
        login_user: "root"
        login_password: ""
        name: "root"
        password: "{{ mysql_root_pwd }}"
        host_all: yes

    - name: "removing anonymous users"
      mysql_user:
        login_user: "root"
        login_password: "{{ mysql_root_pwd }}"
        name: ""
        host_all: yes
        state: absent
    
    - name: "Creating wordpress database {{ mysql_extra_db }}"
      mysql_db:
        login_password: "{{ mysql_root_pwd }}"
        login_user: "root"
        name: "{{ mysql_extra_db }}"
        state: present

    - name: "Creating wordpress user {{ mysql_extra_user }}"
      mysql_user:
        login_user: "root"
        login_password: "{{ mysql_root_pwd }}"
        name: "{{ mysql_extra_user }}"
        host: "%"
        priv: "{{ mysql_extra_db }}.*:ALL"

In [ ]:
# ansible-playbook -i hosts wordpress.yml 

PLAY [Installing and configuring mariadb server for wordpress] ********************************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
ok: [172.31.37.108]

TASK [installing mariadb and pip packages] ****************************************************************************************************************
ok: [172.31.37.108]

TASK [pip] ************************************************************************************************************************************************
ok: [172.31.37.108]

TASK [Restarting/enabling mariadb server] *****************************************************************************************************************
changed: [172.31.37.108]

TASK [Resetting root password] ****************************************************************************************************************************
[WARNING]: Module did not set no_log for update_password
fatal: [172.31.37.108]: FAILED! => {"changed": false, "msg": "unable to connect to database, check login_user and login_password are correct or /root/.my.cnf has the credentials. Exception message: (1045, u\"Access denied for user 'root'@'localhost' (using password: NO)\")"}
...ignoring

TASK [removing anonymous users] ***************************************************************************************************************************
ok: [172.31.37.108]

TASK [Creating wordpress database wordpressdb] ************************************************************************************************************
changed: [172.31.37.108]

TASK [Creating wordpress user wpuser] *********************************************************************************************************************
changed: [172.31.37.108]

PLAY RECAP ************************************************************************************************************************************************
172.31.37.108              : ok=8    changed=3    unreachable=0    failed=0    skipped=0    rescued=0    ignored=1 

### apache and php installation on amazon linux

In [ ]:
# cat apache.vars 
---
wp_url: "https://wordpress.org/latest.tar.gz"
apache_user: "apache"
apache_group: "apache"
domain_name: "blog.inenso.in"
[root@ip-172-31-36-60 ec2-user]# cat mariadb-wordpress.vars 
---
mysql_root_pwd: "mysqlroot@123*"
mysql_extra_user: "wpuser"
mysql_extra_pwd: "wppwd@123*"
mysql_extra_db: "wordpressdb"

In [ ]:
# cat conf/httpd.conf |grep -i include
Include conf.modules.d/*.conf
IncludeOptional conf.d/*.conf


In [ ]:
# vim vhost.tmpl

<VirtualHost *:80>
    ServerName {{ domain_name }}
    DocumentRoot /var/www/{{ domain_name }}
    directoryindex index.php index.html
    <directory /var/www/{{ domain_name }}>
       allowoverride all
    </directory>
</VirtualHost>

In [ ]:
# copying info.php and info.html and apache confgration file for domain

In [ ]:
# cat wordpress.yml 
---
- name: "Installing and configuring mariadb server with wordpress(LAMP)"
  hosts: amazon
  become: yes
  vars_files:
    - /home/ec2-user/mariadb-wordpress.vars
    - /home/ec2-user/apache.vars
  tasks:
    - name: "installing mariadb and pip packages"
      yum: 
        name: 
          - mariadb-server
          - pip 
        state: present
    - name:
      pip:
        name: PyMySQL
        state: present

    - name: "Restarting/enabling mariadb server"
      service:
        name: mariadb
        state: restarted
        enabled: yes    

    - name: "Resetting root password"
      ignore_errors: yes
      mysql_user:
        login_user: "root"
        login_password: ""
        name: "root"
        password: "{{ mysql_root_pwd }}"
        host_all: yes

    - name: "removing anonymous users"
      mysql_user:
        login_user: "root"
        login_password: "{{ mysql_root_pwd }}"
        name: ""
        host_all: yes
        state: absent
    
    - name: "Creating wordpress database {{ mysql_extra_db }}"
      mysql_db:
        login_password: "{{ mysql_root_pwd }}"
        login_user: "root"
        name: "{{ mysql_extra_db }}"
        state: present

    - name: "Creating wordpress user {{ mysql_extra_user }}"
      mysql_user:
        login_user: "root"
        login_password: "{{ mysql_root_pwd }}"
        name: "{{ mysql_extra_user }}"
        password: "{{ mysql_extra_pwd }}"
        host: "%"
        priv: "{{ mysql_extra_db }}.*:ALL"
        

    - name: "apache package installation"
      yum:
        name: "httpd"
        state: present
      
    - name: "php installation on amazon linux"
      shell:
        cmd: "amazon-linux-extras install php7.4 -y"

    - name: "creating document root /var/www/{{ domain_name }}/"
      file:
        state: directory
        group: "{{ apache_user }}"
        owner: "{{ apache_user }}"
        path: "/var/www/{{ domain_name }}"

    - name: "copying files to /var/www/{{ domain_name }}/"
      copy:
        src: "{{ item }}"
        dest: "/var/www/{{ domain_name }}"
        owner: "{{ apache_user }}"
        group: "{{ apache_group }}"
      with_items:
        - info.php
        - info.html

    - name: "configuration file for domain {{ domain_name }} at /etc/httpd/conf.d/{{ domain_name }}.conf"
      template:  
        src: "/home/ec2-user/vhost.tmpl"
        dest: "/etc/httpd/conf.d/{{ domain_name }}.conf"
      register: vhost_status 
     
      
    - name: "restart/enable php-fpm and httpd"
      when: vhost_status.changed == true
      service:
        name: "{{ item }}"
        state: restarted
        enabled: yes
      with_items:
        - httpd
        - php-fpm


In [ ]:
# ansible-playbook -i hosts wordpress.yml 

PLAY [Installing and configuring mariadb server with wordpress(LAMP)] *************************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
[WARNING]: Platform linux on host 172.31.37.108 is using the discovered Python interpreter at /usr/bin/python, but future installation of another Python
interpreter could change this. See https://docs.ansible.com/ansible/2.9/reference_appendices/interpreter_discovery.html for more information.
ok: [172.31.37.108]

TASK [installing mariadb and pip packages] ****************************************************************************************************************
ok: [172.31.37.108]

TASK [pip] ************************************************************************************************************************************************
ok: [172.31.37.108]

TASK [Restarting/enabling mariadb server] *****************************************************************************************************************
changed: [172.31.37.108]

TASK [Resetting root password] ****************************************************************************************************************************
[WARNING]: Module did not set no_log for update_password
fatal: [172.31.37.108]: FAILED! => {"changed": false, "msg": "unable to connect to database, check login_user and login_password are correct or /root/.my.cnf has the credentials. Exception message: (1045, u\"Access denied for user 'root'@'localhost' (using password: NO)\")"}
...ignoring

TASK [removing anonymous users] ***************************************************************************************************************************
ok: [172.31.37.108]

TASK [Creating wordpress database wordpressdb] ************************************************************************************************************
ok: [172.31.37.108]

TASK [Creating wordpress user wpuser] *********************************************************************************************************************
ok: [172.31.37.108]

TASK [apache package installation] ************************************************************************************************************************
ok: [172.31.37.108]

TASK [php installation on amazon linux] *******************************************************************************************************************
changed: [172.31.37.108]

TASK [creating document root /var/www/blog.inenso.in/] ****************************************************************************************************
changed: [172.31.37.108]

TASK [copying files to /var/www/blog.inenso.in/] **********************************************************************************************************
changed: [172.31.37.108] => (item=info.php)
changed: [172.31.37.108] => (item=info.html)

TASK [configuration file for domain blog.inenso.in at /etc/httpd/conf.d/blog.inenso.in.conf] **************************************************************
changed: [172.31.37.108]

TASK [restart/enable php-fpm and httpd] *******************************************************************************************************************
changed: [172.31.37.108] => (item=httpd)
changed: [172.31.37.108] => (item=php-fpm)

PLAY RECAP ************************************************************************************************************************************************
172.31.37.108              : ok=14   changed=6    unreachable=0    failed=0    skipped=0    rescued=0    ignored=1  

### downloading,extracting,copying,congig file updating of wordpress

In [ ]:
# cat apache.vars 
---
wp_url: "https://wordpress.org/latest.tar.gz"
apache_user: "apache"
apache_group: "apache"
domain_name: "blog.inenso.in"
[root@ip-172-31-36-60 ec2-user]# cat  mariadb-wordpress.vars 
---
mysql_root_pwd: "mysqlroot@123*"
mysql_extra_user: "wpuser"
mysql_extra_pwd: "wppwd@123*"
mysql_extra_db: "wordpressdb"


In [ ]:
# cat wp-config-sample.php 
<?php


// ** Database settings - You can get this info from your web host ** //
/** The name of the database for WordPress */
define( 'DB_NAME', '{{ mysql_extra_db }}' );

/** Database username */
define( 'DB_USER', '{{ mysql_extra_user }}' );

/** Database password */
define( 'DB_PASSWORD', '{{ mysql_extra_pwd }}' );

/** Database hostname */
define( 'DB_HOST', 'localhost' );


In [ ]:
# cat wordpress.yml 
---
- name: "Installing and configuring mariadb server with wordpress(LAMP)"
  hosts: amazon
  become: yes
  vars_files:
    - mariadb-wordpress.vars
    - apache.vars
  tasks:
    - name: "installing mariadb and pip packages"
      yum: 
        name: 
          - mariadb-server
          - pip 
        state: present
    - name:
      pip:
        name: PyMySQL
        state: present

    - name: "Restarting/enabling mariadb server"
      service:
        name: mariadb
        state: restarted
        enabled: yes    

    - name: "Resetting root password"
      ignore_errors: yes
      mysql_user:
        login_user: "root"
        login_password: ""
        name: "root"
        password: "{{ mysql_root_pwd }}"
        host_all: yes

    - name: "removing anonymous users"
      mysql_user:
        login_user: "root"
        login_password: "{{ mysql_root_pwd }}"
        name: ""
        host_all: yes
        state: absent
    
    - name: "Creating wordpress database {{ mysql_extra_db }}"
      mysql_db:
        login_password: "{{ mysql_root_pwd }}"
        login_user: "root"
        name: "{{ mysql_extra_db }}"
        state: present

    - name: "Creating wordpress user {{ mysql_extra_user }}"
      mysql_user:
        login_user: "root"
        login_password: "{{ mysql_root_pwd }}"
        name: "{{ mysql_extra_user }}"
        password: "{{ mysql_extra_pwd }}"
        host: "%"
        priv: "{{ mysql_extra_db }}.*:ALL"
        

    - name: "apache package installation"
      yum:
        name: "httpd"
        state: present
      
    - name: "php installation on amazon linux"
      shell:
        cmd: "amazon-linux-extras install php7.4 -y"

    - name: "creating document root /var/www/{{ domain_name }}/"
      file:
        state: directory
        group: "{{ apache_user }}"
        owner: "{{ apache_user }}"
        path: "/var/www/{{ domain_name }}"

    - name: "copying files to /var/www/{{ domain_name }}/"
      copy:
        src: "{{ item }}"
        dest: "/var/www/{{ domain_name }}"
        owner: "{{ apache_user }}"
        group: "{{ apache_group }}"
      with_items:
        - info.php
        - info.html

    - name: "configuration file for domain {{ domain_name }} at /etc/httpd/conf.d/{{ domain_name }}.conf"
      template:  
        src: "/home/ec2-user/vhost.tmpl"
        dest: "/etc/httpd/conf.d/{{ domain_name }}.conf"
      register: vhost_status 
     
      
    - name: "restart/enable php-fpm and httpd"
      when: vhost_status.changed == true
      service:
        name: "{{ item }}"
        state: restarted
        enabled: yes
      with_items:
        - httpd
        - php-fpm 
     
   
    - name: "Downloading wordpress"
      get_url:
        dest: /tmp/wordpress.tar.gz
        url: "{{ wp_url}}"

    - name: "extracting the wordpress core files"
      unarchive:
        dest: /tmp/
        src: /tmp/wordpress.tar.gz
        remote_src: yes

    - name: "copying wordpress files to document root /var/www/{{ domain_name }}"
      copy:
        src: /tmp/wordpress/
        dest: "/var/www/{{ domain_name }}"
        remote_src: yes
        owner: "{{ apache_user }}"
        group: "{{ apache_group }}"

    - name: "creating wp-config.php from template"
      template:
        src: wp-config-sample.php
        dest: "/var/www/{{ domain_name }}/wp-config.php"
        owner: "{{ apache_user }}"
        group: "{{ apache_group }}"

    - name: "post installation cleanup"
      file:
        path: "{{ item }}"
        state: absent
      with_items:
        - /tmp/wordpress.tar.gz
        - /tmp/wordpress

In [ ]:
# ansible-playbook -i hosts wordpress.yml 

PLAY [Installing and configuring mariadb server with wordpress(LAMP)] *************************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
[WARNING]: Platform linux on host 172.31.37.108 is using the discovered Python interpreter at /usr/bin/python, but future installation of another Python
interpreter could change this. See https://docs.ansible.com/ansible/2.9/reference_appendices/interpreter_discovery.html for more information.
ok: [172.31.37.108]

TASK [installing mariadb and pip packages] ****************************************************************************************************************
ok: [172.31.37.108]

TASK [pip] ************************************************************************************************************************************************
ok: [172.31.37.108]

TASK [Restarting/enabling mariadb server] *****************************************************************************************************************
changed: [172.31.37.108]

TASK [Resetting root password] ****************************************************************************************************************************
[WARNING]: Module did not set no_log for update_password
fatal: [172.31.37.108]: FAILED! => {"changed": false, "msg": "unable to connect to database, check login_user and login_password are correct or /root/.my.cnf has the credentials. Exception message: (1045, u\"Access denied for user 'root'@'localhost' (using password: NO)\")"}
...ignoring

TASK [removing anonymous users] ***************************************************************************************************************************
ok: [172.31.37.108]

TASK [Creating wordpress database wordpressdb] ************************************************************************************************************
ok: [172.31.37.108]

TASK [Creating wordpress user wpuser] *********************************************************************************************************************
ok: [172.31.37.108]

TASK [apache package installation] ************************************************************************************************************************
ok: [172.31.37.108]

TASK [php installation on amazon linux] *******************************************************************************************************************
changed: [172.31.37.108]

TASK [creating document root /var/www/blog.inenso.in/] ****************************************************************************************************
ok: [172.31.37.108]

TASK [copying files to /var/www/blog.inenso.in/] **********************************************************************************************************
ok: [172.31.37.108] => (item=info.php)
ok: [172.31.37.108] => (item=info.html)

TASK [configuration file for domain blog.inenso.in at /etc/httpd/conf.d/blog.inenso.in.conf] **************************************************************
ok: [172.31.37.108]

TASK [restart/enable php-fpm and httpd] *******************************************************************************************************************
skipping: [172.31.37.108] => (item=httpd) 
skipping: [172.31.37.108] => (item=php-fpm) 

TASK [Downloading wordpress] ******************************************************************************************************************************
changed: [172.31.37.108]

TASK [extracting the wordpress core files] ****************************************************************************************************************
changed: [172.31.37.108]

TASK [copying wordpress files to document root /var/www/blog.inenso.in] ***********************************************************************************
ok: [172.31.37.108]

TASK [creating wp-config.php from template] ***************************************************************************************************************
ok: [172.31.37.108]

TASK [post installation cleanup] **************************************************************************************************************************
changed: [172.31.37.108] => (item=/tmp/wordpress.tar.gz)
changed: [172.31.37.108] => (item=/tmp/wordpress)

PLAY RECAP ************************************************************************************************************************************************
172.31.37.108              : ok=18   changed=5    unreachable=0    failed=0    skipped=1    rescued=0    ignored=1 

In [ ]:
# ansible-playbook -i hosts wordpress.yml --list-tasks

playbook: wordpress.yml

  play #1 (amazon): Installing and configuring mariadb server with wordpress(LAMP)	TAGS: []
    tasks:
      installing mariadb and pip packages	TAGS: []
      pip	TAGS: []
      Restarting/enabling mariadb server	TAGS: []
      Resetting root password	TAGS: []
      removing anonymous users	TAGS: []
      Creating wordpress database {{ mysql_extra_db }}	TAGS: []
      Creating wordpress user {{ mysql_extra_user }}	TAGS: []
      apache package installation	TAGS: []
      php installation on amazon linux	TAGS: []
      creating document root /var/www/{{ domain_name }}/	TAGS: []
      copying files to /var/www/{{ domain_name }}/	TAGS: []
      configuration file for domain {{ domain_name }} at /etc/httpd/conf.d/{{ domain_name }}.conf	TAGS: []
      restart/enable php-fpm and httpd	TAGS: []
      Downloading wordpress	TAGS: []
      extracting the wordpress core files	TAGS: []
      copying wordpress files to document root /var/www/{{ domain_name }}	TAGS: []
      creating wp-config.php from template	TAGS: []
      post installation cleanup	TAGS: []
